# PA6: Sentiment Classification 
##### By: Jon Llamado | NLP1000, XX22


## Description

## Data Pre-processing

In [1]:
import pandas as pd
import re

from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
train_data = pd.read_csv('train.csv')
train_data.head()

,DocumentId,Text,IsPositive
0,2388179,never again LINK,True
1,657251,ma okey rang tanan promise naas lord dita nya ...,True
2,1730789,MENTION yawaaa gyud oh ?,True
3,868789,"bahalag mangalata ang all, dili jud ko mag gma...",False
4,1570427,makatambok gyud diay ang wholeday na tulog,False


In [3]:
test_data = pd.read_csv('test.csv')
test_data.head()

,DocumentId,Text
0,1450521,hi baby boy LINK
1,889575,MENTION hahahaha. nice ra tan awon pero patay ...
2,1168201,nadagdagan ang tropa simula nung napunta ako s...
3,81808,"sooo, i started cleaning my old room and found..."
4,188577,"and stop calling it ""good government"" dahil sa..."


In [36]:
text_array = train_data['Text'].to_numpy()

# Alternatively, you can use the .values attribute
# text_array = df['Text'].values

print("Text column as NumPy array:", text_array)

TypeError: list indices must be integers or slices, not str

In [32]:
X_train

<bound method IndexOpsMixin.to_numpy of 0                                         never again  LINK
1         ma okey rang tanan promise naas lord dita nya ...
2                                  MENTION yawaaa gyud oh ?
3         bahalag mangalata ang all, dili jud ko mag gma...
4               makatambok gyud diay ang wholeday na tulog 
                                ...                        
105840                  MENTION dr. strangers na sad, ma'am
105841             MENTION good luck sa kabataan ng balite 
105842     papa beke nemen buy moko st. ives skin care set 
105843    wag tayo maparanoid sa mga tweets ni ate max. ...
105844    congratulations babies!!!  so proud  #bts #bts...
Name: Text, Length: 105845, dtype: object>

In [26]:
corpus = []

train_data = re.findall('[^a-zA-Z]', X_train[0])
train_data

TypeError: 'method' object is not subscriptable

In [16]:
corpus

['neveragainlink',
 'maokeyrangtananpromisenaaslordditanyapasagdanmagkinaunsaman',
 'mentionyawaaagyudoh',
 'bahalagmangalataangalldilijudkomaggmaaanooonotoabscbnshutdown',
 'makatambokgyuddiayangwholedaynatulog',
 'grabetumawadmganaorderhahahasisyungtubokodiyanlang',
 'goodmorningmgasawii',
 'stormytuesdaybravingpepitokeepsafeanddrysweaterjeansandmylinkquezoncityphilippineslink',
 'aaahhhsuccessyungpagconvincekokaymamaatbibilhannaniyaakongbagongphoneatlast',
 'asmartvillainpersebutactingstupidandchildishitsanofromus',
 'timecheckammedyonakakaramdamnaakongantokkontingorasnalangmakakatulogdinakomorebookingtrendslink',
 'lastnajudnioooyyypleaselastnajudninglinkkoookaykaaapoooyyynajudkaayooo',
 'solidangtambay',
 'chuyaypagyugdesignstananpatisoyba',
 'tanginaaasananamanfakenewslangyunlordpleaseeekeepeveryonesafeespeciallymyfamily',
 'bermonthsiscomingmalungkotnapaskoiswaving',
 'tagagokinukuhaacckosamlaybakanahihilokabeh',
 'ughhhicantsleep',
 'hoybatganyankahuhulink',
 'mentionoototoonam

In [12]:
cv = CountVectorizer(max_features = 1500)
 
X = cv.fit_transform(corpus).toarray()
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

## Feature Extraction

## Training the Model

##### Divide the training dataset into X_train and y_train


In [7]:
X_train = train_data["Text"]
X_train

0                                         never again  LINK
1         ma okey rang tanan promise naas lord dita nya ...
2                                  MENTION yawaaa gyud oh ?
3         bahalag mangalata ang all, dili jud ko mag gma...
4               makatambok gyud diay ang wholeday na tulog 
                                ...                        
105840                  MENTION dr. strangers na sad, ma'am
105841             MENTION good luck sa kabataan ng balite 
105842     papa beke nemen buy moko st. ives skin care set 
105843    wag tayo maparanoid sa mga tweets ni ate max. ...
105844    congratulations babies!!!  so proud  #bts #bts...
Name: Text, Length: 105845, dtype: object

In [8]:
y_train = train_data["IsPositive"]
y_train

0          True
1          True
2          True
3         False
4         False
          ...  
105840     True
105841     True
105842     True
105843     True
105844     True
Name: IsPositive, Length: 105845, dtype: bool

In [9]:
print(X_train.shape)
print(y_train.shape)

(105845,)
(105845,)


In [10]:
naive_bayes = GaussianNB()

In [11]:
naive_bayes.fit(X_train, y_train)

ValueError: could not convert string to float: 'never again  LINK'

## Testing the Model

## Insights and Conclusions
